<a href="https://colab.research.google.com/github/jux310/CombinadorUade/blob/main/Uade_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uade Scrapper

Este programa simplifica la extracción y filtrado de materias, permitiendo una integración directa con herramientas de planificación de horarios.

#### URL Principal:
Es la dirección de la página web desde donde se extraen los datos de las materias.
Ejemplo: La página institucional de UADE.

```
url = "https://www.webcampus.uade.edu.ar/MiProgramacion...
```

#### HTM Filtro:
Este filtro es para quitar las materias que estan bloqueadas por correlativas, las que no estan disponibles en el proximo periodo y las que no tienen cupos. Este archivo se obtiene entando a inscripciones.uade.edu.ar -> asignaturas cuatrimestre -> Seleccione sus materias y haciendo click derecho/guardar como sobre la tabla de materias. Se obtiene un archivo "InscripcionClaseBuscar.htm" el cual se sube al programa.

```
<Subir archivo "InscripcionClaseBuscar.htm">
```

#### Salida:
El programa genera un archivo CSV llamado materias_completas el cual se descarga automaticamente.

```
Descargando materias_completas.csv
```

El CSV resultante se puede cargar en la sección de preferencias en https://uade.netlify.app para generar combinaciones de horarios a partir de las materias obtenidas.



-



In [80]:
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import difflib  # Para la comparación aproximada
from google.colab import files

print("Obtenga los datos necesarios desde https://inscripciones.uade.edu.ar/\n")
url = input('Ingrese URL de Disponibilidad de clase:\n')
print("")

uploaded = files.upload()
html_filename = next(iter(uploaded))

if html_content_uploaded:
    soup = BeautifulSoup(html_content_uploaded, 'html.parser')
    materia_tables = soup.find_all('table', class_='gridViewMaterias')
    years = [span.text for span in soup.find_all('span', class_='anio')]
    all_materia_data = []

    for year_index, table in enumerate(materia_tables):
        headers = [th.text.replace("<br>", " ").strip() for th in table.find('thead').find_all('th')] if table.find('thead') else [th.text.replace("<br>", " ").strip() for th in table.find('tr').find_all('th')]
        rows = table.find('tbody').find_all('tr') if table.find('tbody') else table.find_all('tr')[1:]

        year_name = years[year_index] if year_index < len(years) else "Año Desconocido"

        for row in rows[1:]:
            cols = row.find_all('td')
            if not cols:
                continue

            materia_data = {}
            for i, col in enumerate(cols):
                header_text = headers[i] if i < len(headers) else f"Columna{i+1}"
                cell_text = col.text.strip()
                materia_data[header_text] = cell_text

            materia_data['Año'] = year_name
            all_materia_data.append(materia_data)

    df = pd.DataFrame(all_materia_data)

    df_filtrado = df[df['Información'] == '']
    lista_nombres_materias = df_filtrado['Materia'].tolist()

else:
    print("\nNo se pudo leer el archivo HTML. Revise si subió el archivo correctamente y si no hubo errores al leerlo.")

http_headers = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                    "AppleWebKit/537.36 (KHTML, like Gecko) "
                    "Chrome/91.0.4472.124 Safari/537.36"),
    "Referer": url
}

def eliminar_tildes(texto):
    return ''.join(
        char for char in unicodedata.normalize('NFKD', str(texto))
        if not unicodedata.combining(char)
    )

def get_form_fields(soup):
    """
    Recolecta todos los campos del formulario, incluyendo inputs hidden y selects.
    """
    fields = {}
    # Inputs hidden
    for input_tag in soup.find_all('input', type='hidden'):
        name = input_tag.get('name')
        if name:
            fields[name] = input_tag.get('value', '')
    # Selects (dropdowns)
    for select_tag in soup.find_all('select'):
        name = select_tag.get('name')
        if name:
            selected_option = select_tag.find('option', selected=True)
            if selected_option:
                fields[name] = selected_option.get('value', '')
            else:
                fields[name] = ''
    return fields

def parse_table(soup):
    """
    Extrae los datos de la tabla.
    Primero intenta buscar la tabla por el id esperado, y si no la encuentra,
    busca directamente en el <tbody>.
    Para las columnas Col9 a Col15 (índices 8 a 14) se asigna True si el estilo contiene
    "background-color:LightGrey", o False en caso contrario.
    Solo se procesan filas que tengan 18 columnas.
    """
    # Intentar encontrar la tabla por id
    table = soup.find('table', {'id': 'ctl00_ContentPlaceHolderMain_RadGrid1_ctl00'})
    if table:
        tbody = table.find('tbody')
    else:
        tbody = soup.find('tbody')

    if not tbody:
        print("No se encontró el <tbody> en el HTML.")
        return [], []

    rows = []
    for row in tbody.find_all('tr'):
        tds = row.find_all('td')
        if not tds:
            continue
        row_data = []
        # Procesar cada celda
        for i, td in enumerate(tds):
            # Para las columnas Col9 a Col15, evaluar el estilo
            if i in range(8, 15):  # índices 8 a 14 (Col9 a Col15)
                style = td.get("style", "")
                if "background-color:LightGrey" in style:
                    row_data.append(True)
                else:
                    row_data.append(False)
            else:
                row_data.append(td.get_text(strip=True))
        if len(row_data) != 18:
            print("Fila con cantidad inesperada de columnas (se esperan 18):", row_data)
            continue  # Se omite la fila si no cumple la cantidad esperada
        rows.append(row_data)

    # Definir manualmente los encabezados (ajusta los nombres según corresponda)
    headers = [
        "Código", "Materia", "Sede", "ID", "Modalidad",
        "Idioma", "Turno", "Año",
        "LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES", "SABADO", "DOMINGO",
        "Horario", "Fechas", "Tipo"
    ]

    return headers, rows

def filter_materias(df, lista_nombres_materias, threshold=0.8): # Modificado para usar lista_nombres_materias
    """
    Filtra el DataFrame 'df' dejando únicamente las materias que sean similares a
    alguna de las materias listadas en  'lista_nombres_materias'.

    Tanto los nombres en lista_nombres_materias como los del DataFrame se normalizan
    (quitando espacios y pasando a minúsculas). Se utiliza difflib para comparar
    de forma aproximada, aceptándose aquellas coincidencias cuyo ratio sea mayor
    o igual a 'threshold'.
    """

    print(f"\nFiltrando materias: se usarán {len(lista_nombres_materias)} materias de la lista proporcionada.")

    # Normalizar la columna "Materia" del DataFrame
    df['Materia'] = df['Materia'].str.replace('"', '').str.replace('"', '').str.replace(',', '')
    df["Materia_norm"] = df["Materia"].astype(str).str.strip().str.lower().apply(eliminar_tildes)

    # Preprocesar lista_nombres_materias: normalizar y pasar a minúsculas
    lista_materias_norm = [eliminar_tildes(materia.strip().lower()) for materia in lista_nombres_materias]


    # Función que verifica si la materia es similar a alguna materia en la lista
    def is_similar(materia):
        for m in lista_materias_norm:
            ratio = difflib.SequenceMatcher(None, materia, m).ratio()
            if ratio >= threshold:
                return True
        return False

    # Aplicar la función a cada fila para determinar si se incluye
    df_filtrado = df[df["Materia_norm"].apply(is_similar)].copy()
    df_filtrado.drop(columns="Materia_norm", inplace=True)
    return df_filtrado

def filter_sede_pinamar(df):
    """
    Filtra el DataFrame 'df' eliminando las filas donde la columna 'Sede'
    contiene la palabra 'PINAMAR' (insensible a mayúsculas/minúsculas).
    """
    print("\nFiltrando sede: eliminando materias de PINAMAR.")
    df_filtrado = df[~df['Sede'].str.contains('PINAMAR', case=False)].copy()
    return df_filtrado


# Iniciar sesión
session = requests.Session()

# Primera carga: obtener parámetros iniciales y HTML completo
response = session.get(url, headers=http_headers)
soup = BeautifulSoup(response.text, 'html.parser')

# (Opcional) Guardar la página inicial para revisar su estructura
with open('pagina_inicial.html', 'w', encoding='utf-8') as f:
    f.write(response.text)

# Obtener los campos del formulario
form_data = get_form_fields(soup)

# Primera solicitud POST con los parámetros obtenidos
response = session.post(url, headers=http_headers, data=form_data)
soup = BeautifulSoup(response.text, 'html.parser')

# Procesar la primera "página" de datos
headers, data = parse_table(soup)
all_data = data.copy()

# Detectar paginación (si existe)
pagination = soup.find('div', class_='rgInfoPart')
total_pages = 1
if pagination:
    match = re.search(r'en (\d+) paginas?\(s\)', pagination.get_text(strip=True))
    if match:
        total_pages = int(match.group(1))

print(f"\nTotal de páginas: {total_pages}")

# Recorrer las páginas restantes (si total_pages > 1)
for page in range(2, total_pages + 1):
    # Actualizar los campos del formulario desde la página actual
    form_data = get_form_fields(soup)

    # Actualizar parámetros para paginación (verificar que __EVENTTARGET sea el correcto)
    form_data.update({
        '__EVENTTARGET': 'ctl00$ContentPlaceHolderMain$RadGrid1$ctl00$ctl03$ctl01$ctl'+str(6+2*total_pages),
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': ''
    })

    # Solicitud POST para la siguiente página
    response = session.post(url, headers=http_headers, data=form_data)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraer datos de la página
    _, page_data = parse_table(soup)
    if page_data:
        all_data.extend(page_data)
    else:
        print(f"No se encontraron filas válidas en la página {page}.")

    print(f"Procesada página {page}/{total_pages}")

# Crear DataFrame con los datos extraídos
if headers and all_data:
    try:
        df = pd.DataFrame(all_data, columns=headers)
        # Aplicar filtro usando lista_nombres_materias
        df = filter_materias(df, lista_nombres_materias, threshold=0.8) # Modificado para usar lista_nombres_materias
        # Aplicar filtro para eliminar la sede PINAMAR
        df = filter_sede_pinamar(df) # Nueva línea para filtrar por sede PINAMAR
        df.to_csv('materias_completas.csv', index=False, encoding='utf-8-sig')
        print("Archivo guardado correctamente: materias_completas.csv")
        files.download('materias_completas.csv')
        print("\nImportar CSV a https://uade.netlify.app/")
    except Exception as e:
        print("Error al crear el DataFrame:", e)
else:
    print("No se encontraron datos válidos")

# Eliminar archivos intermedios
!rm *.htm
!rm *.html

Obtenga los datos necesarios desde https://inscripciones.uade.edu.ar/

Ingrese URL de Disponibilidad de clase:
https://www.webcampus.uade.edu.ar/MiProgramacion/Contenidos/MiProgramacion.aspx?IdUsuario=br+B3VmrZbw=



Saving InscripcionClaseBuscar.htm to InscripcionClaseBuscar.htm

Total de páginas: 3
Procesada página 2/3
Procesada página 3/3

Filtrando materias: se usarán 5 materias de la lista proporcionada.

Filtrando sede: eliminando materias de PINAMAR.
Archivo guardado correctamente: materias_completas.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Importar CSV a https://uade.netlify.app/
